In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd 'drive/My Drive/Major Project (SEM-7)/'

/content/drive/.shortcut-targets-by-id/1TQWlNRpi0pg1yJzANQ4DdxR7TDTDyAcd/Major Project (SEM-7)


In [ ]:
import numpy as np
import os
import time
# from resnet50 import ResNet50
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten

# from imagenet_utils import preprocess_input
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split #cross-validation

In [ ]:
PATH = os.getcwd()
# Define data path
data_path = PATH + '/data2'
data_dir_list = os.listdir(data_path)

In [ ]:
def preprocess_input(x):
  return x

In [ ]:
import cv2

img_data_list=[]

for dataset in data_dir_list:
  img_list=os.listdir(data_path+'/'+ dataset)
  print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
  for img in img_list:
    img_path = data_path + '/'+ dataset + '/'+ img
    image = cv2.imread(img_path)
    image = cv2.resize(image, (256, 256)) 
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    lab_planes = cv2.split(lab) 

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab_planes[1] = clahe.apply(lab_planes[1])
    lab_planes[2] = clahe.apply(lab_planes[2])


    final_img = cv2.merge(lab_planes)
    final_img = cv2.cvtColor(final_img, cv2.COLOR_LAB2RGB)

    # img = img.load_img(img_path, target_size=(224, 224))
    # x = img.img_to_array(img)
    # x = np.expand_dims(x, axis=0)
		# x = preprocess_input(x)
    # print('Input image shape:', x.shape)
    img_data_list.append(final_img)
    print(final_img.shape)

In [ ]:
cd ../../..

/content/drive


In [ ]:
cd "MyDrive/Images Pickles"

/content/drive/MyDrive/Images Pickles


In [ ]:
import pickle
# with open('images_256x256.pkl', 'wb') as f:
#   pickle.dump(img_data_list, f)

k-fold cross Validation for multi class


In [ ]:
import pickle 
with open('images_256x256.pkl', 'rb') as f:
  img_data_list = pickle.load(f)

In [ ]:
# for i in range(len(img_data_list)):
#   img_data_list[i] = cv2.cvtColor(img_data_list[i],cv2.COLOR_GRAY2RGB)
  # b, g, r = cv2.split(img_data_list[i])
  # b = np.zeros((224, 224))
  # g= np.zeros((224,224))
  # img_data_list[i] = cv2.merge((b,g,r))
  # print(b, g, r)
  # break

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
print (img_data.shape)
# img_data=np.rollaxis(img_data,1,0)
# print (img_data.shape)
# img_data=img_data[0]
# img_data=img_data[:1125]
# print (img_data.shape)

(1125, 256, 256, 3)


In [ ]:
# Define the number of classes
num_classes = 3
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

In [ ]:
labels[0:125]=0
labels[125:625]=1
labels[625:1125]=2

In [ ]:
names = ['covid-19','no-findings', 'pneumonia']
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)
# Y=labels

In [ ]:
#Shuffle the dataset
x,y = shuffle(img_data[:1125],Y, random_state=5)
# Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
kf.get_n_splits(x)

5

In [ ]:
print(kf)

KFold(n_splits=5, random_state=None, shuffle=False)


In [ ]:
scores = []

In [ ]:
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils import multi_gpu_model
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.applications.resnet_v2 import ResNet50V2
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Flatten


def build_Classifier():
  for_pad = lambda s: s if s > 2 else 3
  channels = 3
  kernel_size = (3, 3)
  model = Sequential()

  model.add(Conv2D(8, kernel_size=3, padding = "same", strides=1, input_shape=(256, 256,3)))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

  model.add(Conv2D(16, (kernel_size[0], kernel_size[1]), padding = "same", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

  model.add(Conv2D(32, kernel_size=3, padding = "same", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
  model.add(Conv2D(16, kernel_size=1, padding = "valid", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())
  model.add(Conv2D(32, kernel_size=3, padding = "same", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

  model.add(Conv2D(64, kernel_size=3, padding = "same", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
  model.add(Conv2D(32, kernel_size=1, padding = "valid", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())
  model.add(Conv2D(64, kernel_size=3, padding = "same", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

  model.add(Conv2D(128, kernel_size=3, padding = "same", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
  model.add(Conv2D(64, kernel_size=1, padding = "valid", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())
  model.add(Conv2D(128, kernel_size=3, padding = "same", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

  model.add(Conv2D(256, kernel_size=3, padding = "same", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
  model.add(Conv2D(128, kernel_size=1, padding = "valid", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())
  model.add(Conv2D(256, kernel_size=3, padding = "same", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
  model.add(Conv2D(128, kernel_size=1, padding = "valid", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(Conv2D(256, kernel_size=1, padding = "valid", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(Conv2D(3, kernel_size=1, padding = "valid", strides=1))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(Flatten())
  # print("Model flattened out to: ", model.output_shape)
  model.add(tf.keras.layers.Dense(3, activation='softmax'))
  
  sched = OneCycle(max_lr=3e-3)
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
  return model

  # base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

  # model = tf.keras.Sequential([
  #     base_model,
  #     tf.keras.layers.Flatten(),
  #     tf.keras.layers.Dense(1024, activation='relu'),
  #     tf.keras.layers.Dense(256, activation='relu'),
  #     tf.keras.layers.Dense(3, activation='softmax')
  # ])
  
  # model.compile(loss='categorical_crossentropy',
  #           optimizer='adam',
  #           metrics=['accuracy']) 
  # return model
sched = OneCycle(max_lr=3e-3)
classifier = KerasClassifier(build_fn = build_Classifier, batch_size = 32, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = x, y = y, cv=5, fit_params={'callbacks': [sched]})
# model = build_Classifier()
# print(model.summary())



Epoch 1/100
29/29 [==============================] - 2s 67ms/step - loss: 0.8739 - accuracy: 0.5911
Epoch 2/100
29/29 [==============================] - 2s 66ms/step - loss: 0.5857 - accuracy: 0.7444
Epoch 3/100
29/29 [==============================] - 2s 67ms/step - loss: 0.5287 - accuracy: 0.7700
Epoch 4/100
29/29 [==============================] - 2s 67ms/step - loss: 0.5156 - accuracy: 0.7733
Epoch 5/100
29/29 [==============================] - 2s 67ms/step - loss: 0.5062 - accuracy: 0.7922
Epoch 6/100
29/29 [==============================] - 2s 67ms/step - loss: 0.4584 - accuracy: 0.8089
Epoch 7/100
29/29 [==============================] - 2s 68ms/step - loss: 0.4781 - accuracy: 0.7989
Epoch 8/100
29/29 [==============================] - 2s 68ms/step - loss: 0.4038 - accuracy: 0.8300
Epoch 9/100
29/29 [==============================] - 2s 67ms/step - loss: 0.4395 - accuracy: 0.8200
Epoch 10/100
29/29 [==============================] - 2s 67ms/step - loss: 0.3675 - accuracy: 0.8489

In [ ]:
accuracies

array([0.79555553, 0.74666667, 0.65777779, 0.79555553, 0.60444444])

In [ ]:
accuracies.mean()

0.7199999928474426

K fold for binary classification

In [ ]:
img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

(1125, 1, 224, 224, 3)
(1, 1125, 224, 224, 3)
(1125, 224, 224, 3)


In [ ]:
# Define the number of classes
num_classes = 2
num_of_samples = img_data.shape[0]
labels = np.ones((625,),dtype='int64')

In [ ]:
labels[0:125]=0
labels[125:625]=1
# labels[625:1125]=2

In [ ]:
names = ['covid-19','no-findings']
# convert class labels to on-hot encoding
# Y = np_utils.to_categorical(labels, num_classes)
Y=labels

In [ ]:
#Shuffle the dataset
x,y = shuffle(img_data[:625],Y, random_state=5)
# Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
kf.get_n_splits(x)

5

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils import multi_gpu_model
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
channels = 3
nb_filters = 32
kernel_size = (2, 2)
# model = Sequential()

'''
First set of three layers
Image size: 256 x 256
nb_filters = 32
kernel_size = (2,2)
'''
pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[224, 224, 3])

pretrained_model.trainable = False

model4 = tf.keras.Sequential([
    pretrained_model,

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model4.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model4.summary())


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_10 (Flatten)         (None, 25088)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 25089     
Total params: 14,739,777
Trainable params: 25,089
Non-trainable params: 14,714,688
_________________________________________________________________
None


In [ ]:
scores = []

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_Classifier():
  pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[224, 224, 3])

  pretrained_model.trainable = False

  model4 = tf.keras.Sequential([
      pretrained_model,

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model4.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model4

classifier = KerasClassifier(build_fn = build_Classifier, batch_size = 32, epochs = 5)
accuracies = cross_val_score(estimator = classifier, X = x, y = y, cv=5)

Epoch 1/5
16/16 [==============================] - 3s 182ms/step - loss: 1.0412 - accuracy: 0.8880
Epoch 2/5
16/16 [==============================] - 2s 119ms/step - loss: 0.0668 - accuracy: 0.9860
Epoch 3/5
16/16 [==============================] - 2s 119ms/step - loss: 7.1264e-04 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 2s 119ms/step - loss: 0.0064 - accuracy: 0.9980
Epoch 5/5
4/4 [==============================] - 2s 420ms/step - loss: 1.6170 - accuracy: 0.9600
Epoch 1/5
16/16 [==============================] - 2s 119ms/step - loss: 3.7555 - accuracy: 0.8060
Epoch 2/5
16/16 [==============================] - 2s 119ms/step - loss: 0.3136 - accuracy: 0.9720
Epoch 3/5
16/16 [==============================] - 2s 119ms/step - loss: 0.2348 - accuracy: 0.9780
Epoch 4/5
16/16 [==============================] - 2s 120ms/step - loss: 0.0795 - accuracy: 0.9900
Epoch 5/5
4/4 [==============================] - 0s 93ms/step - loss: 0.2880 - accuracy: 0.9760
Epoch 1/5
1

In [ ]:
accuracies

array([0.95999998, 0.97600001, 0.95200002, 0.97600001, 0.99199998])

In [ ]:
accuracies.mean()

0.9712000012397766

In [ ]:
ls

 171IT125/
'171IT125 (1)'/
 171IT125_CBT_Assignment1.zip
 171IT125_Madhumitha/
'171IT125_Madhumitha (1)'/
 17IT125/
'17IT125 (1)'/
'2dgrids2dblocks (1).pdf'
 2dgrids2dblocks.pdf
'3 Sem'/
'4 Sem'/
 _91171624a1133362ea12dc427ce0ac17_C4W1L01-ComputerVision.gslides
 _a5534fb24339aba859f883dd8f69b1fc_C2W3L09.gslides
 Android.gdoc
 AppLayoutforRead.png
 AppLayout.png
 Assessment.gform
'Assignment I.gdoc'
'Attendance for Image Recog using CNNs SMP.gsheet'
 Attendance.gsheet
'Bank Account Details for Inter NIT Jaipur Chess.gdoc'
'Bank Account Details.gdoc'
 BlocksForOnOff.png
'CBT_CrowdFunding (1).mp4'
 CBT_CrowdFunding.mp4
'CBT Syllabus.gdoc'
'CG Notes'/
 ChannelValuesAfterButtonPress.png
'Cluster Results.gsheet'
 Code.zip
'coherenceprotocols (1).pdf'
 coherenceprotocols.pdf
'Colab Notebooks'/
'Competitive Programming.gdoc'
'CompilerOptimizationsnotes (1).pdf'
 CompilerOptimizationsnotes.pdf
 Computer-Organization-by-Carl-Hamacher-5th.pdf
'Copy of DGCAN.py'
'Copy of HTML Form Reponses.gsheet'

In [ ]:
cd COVIDNet-CXR-Large/

/content/drive/.shortcut-targets-by-id/1eNidqMyz3isLjGYN1evzQu--A-JVkzbk/COVIDNet-CXR-Large


In [ ]:
new_model = tf.keras.models.load_model('saved_model')

OSError: ignored

In [ ]:
cd COVIDNet-CXR-Large/

/content/drive/MyDrive


In [ ]:
from keras.callbacks import Callback
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

#####################
##### OneCycle  #####
#####################

          
class OneCycle(Callback):
    """This callback implements a cyclical learning rate and momentum policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration 
    For more detail, please see paper.
    
    # Example
        ```python
            clr = OneCycle(min_lr=1e-3, max_lr=1e-2,
                      min_mtm=0.85, max_mtm=0.95,
                      annealing=0.1,step_size=np.ceil((X_train.shape[0]*epochs/batch_size)))
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    # Arguments
        min_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - min_lr).
        step_size: number of training iterations in the cycle. To define as `np.ceil((X_train.shape[0]*epochs/batch_size))`
        max_mtm : initial value of the momentum    
        min_mtm : lower boundary in the cycle.
        annealing_stage : percentage of the iterations where the lr
                    will decrease lower than its min_lr
        annealing_rate : in annealing phase learning rate will be decreased to annealing_rate*min_lr
                    
        # References
        Original paper: https://arxiv.org/pdf/1803.09820.pdf
        Inspired by : https://sgugger.github.io/the-1cycle-policy.html#the-1cycle-policy
    """

    def __init__(self, min_lr=1e-5, max_lr=1e-2, min_mtm = 0.85, max_mtm=0.95, training_iterations=1000.,
                 annealing_stage=0.1, annealing_rate=0.01):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.min_mtm = min_mtm
        self.max_mtm = max_mtm
        self.annealing_stage = annealing_stage
        self.step_size = training_iterations*(1-self.annealing_stage)/2
        self.min_annealing_lr = annealing_rate * min_lr
        self.iterations = 0.
        self.training_iterations = training_iterations
        self.history = {}
        
    def clr(self):
        if self.iterations < 2*self.step_size :
            x = np.abs(self.iterations/self.step_size - 1)
            return self.min_lr + (self.max_lr-self.min_lr)*(1-x)
        else:
            x = min(1, float(self.iterations - 2 * self.step_size) / (self.training_iterations - 2 * self.step_size))
            return self.min_lr - (self.min_lr - self.min_annealing_lr) * x
        
    
    def cmtm(self):
        if self.iterations < 2*self.step_size :   
            x = np.abs(self.iterations/self.step_size - 1)
        else: 
            x=1
        return self.min_mtm + (self.max_mtm-self.min_mtm)*(x)     
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        K.set_value(self.model.optimizer.momentum, self.max_mtm)
         
    def on_batch_end(self, batch, logs=None):
        
        logs = logs or {}
        self.iterations += 1
    
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('momentum', []).append(K.get_value(self.model.optimizer.momentum))
        self.history.setdefault('iterations', []).append(self.iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr()) 
        K.set_value(self.model.optimizer.momentum, self.cmtm())
        
    def plot_lr(self):
        plt.xlabel('Training Iterations')
        plt.ylabel('Learning Rate')
        plt.title("CLR - '1 cycle' Policy")
        plt.plot(self.history['iterations'], self.history['lr'])
        
    def plot_mtm(self):
        plt.xlabel('Training Iterations')
        plt.ylabel('Momentum')
        plt.title("CLR - '1 cycle' Policy")
        plt.plot(self.history['iterations'], self.history['momentum'])

In [1]:
from fastai.vision import *